In [54]:
import pandas as pd

In [55]:
df = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\МКШ\base 141 (134).xlsx", dtype={'iin' : str}).rename(columns={
    'school_ru' : 'school',
    'fio' : 'name',
    'dich' : 'answers',
})

## Data processing

In [56]:
df = df.loc[:, ['school', 'grade', 'subject', 'lang', 'iin', 'answers', 'score']]

In [57]:
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [58]:
df['subject'] = df['subject'].str[2:].apply(lambda x: x[:-1] if ('1' in x or '2' in x) else x)

In [59]:
lang_map = {
    1 : 'Казахский',
    2 : 'Русский'
}
df['lang'] = df['lang'].map(lang_map)

In [60]:
subject_map = {
    'SCIE' : 'Естествознание и познание мира', 
    'PHYS' : 'Физика', 
    'ENFL' : 'Английский язык', 
    'MATH' : 'Математика', 
    'BIOL' : 'Биология', 
    'CHEM' : 'Химия'
}
df['subject'] = df['subject'].map(subject_map)

In [61]:
df['grade'] = df['grade'].astype(str)

## Calculations

In [62]:
df['percent'] = df.apply(lambda row: row['score'] / 20 * 100, axis=1)

In [63]:
g_school = df.groupby('school')['percent'].mean().reset_index().round(1)
g_school

,school,percent
0,ГУ «Общеобразовательная средняя школа имени Б....,38.0
1,"КГУ ""Абайская основная школа""",47.6
2,"КГУ ""Акайская средняя школа""",35.2
3,"КГУ ""Аккудукская средняя школа""",31.2
4,"КГУ ""Акрабская средняя школа""",37.0
...,...,...
136,КГУ «Основная школа имени А.Молдагуловой»,33.1
137,КГУ «Саратская общеобразовательная средняя школа»,44.5
138,КГУ «Теренсайская общеобразовательная средняя ...,44.0
139,КГУ «Хазретовская общеобразовательная средняя ...,53.3


In [64]:
g_school_subject = df.groupby(['school', 'subject'])['percent'].mean().reset_index().round(1)
g_school_subject

,school,subject,percent
0,ГУ «Общеобразовательная средняя школа имени Б....,Английский язык,28.8
1,ГУ «Общеобразовательная средняя школа имени Б....,Биология,46.6
2,ГУ «Общеобразовательная средняя школа имени Б....,Естествознание и познание мира,30.0
3,ГУ «Общеобразовательная средняя школа имени Б....,Математика,27.1
4,ГУ «Общеобразовательная средняя школа имени Б....,Физика,44.3
...,...,...,...
841,КГУ «Школа-сад имени Н.Байганина»,Биология,37.2
842,КГУ «Школа-сад имени Н.Байганина»,Естествознание и познание мира,30.0
843,КГУ «Школа-сад имени Н.Байганина»,Математика,29.6
844,КГУ «Школа-сад имени Н.Байганина»,Физика,32.2


Splitting answers into columns

In [65]:
import numpy as np

cols = np.arange(1,21)

df[cols] = df['answers'].apply(lambda x: pd.Series(list(x)))

In [66]:
df[cols] = df[cols].astype(int)
df[cols] = df[cols].map(lambda x: x - 9 if x == 9 else x)

In [67]:
df

,school,grade,subject,lang,iin,answers,score,percent,1,2,...,11,12,13,14,15,16,17,18,19,20
0,"КГУ ""Общеобразовательная средняя школа имени А...",6,Естествознание и познание мира,Казахский,111031505028,01100000000101001010,6,30.0,0,1,...,0,1,0,1,0,0,1,0,1,0
1,"КГУ ""Общеобразовательная средняя школа имени А...",6,Естествознание и познание мира,Казахский,111225600781,00010000000001110009,4,20.0,0,0,...,0,0,0,1,1,1,0,0,0,0
2,"КГУ ""Общеобразовательная средняя школа имени А...",6,Естествознание и познание мира,Казахский,120105600946,10011011001001001000,8,40.0,1,0,...,1,0,0,1,0,0,1,0,0,0
3,"КГУ ""Общеобразовательная средняя школа имени А...",6,Естествознание и познание мира,Казахский,120113501733,01010001900010100110,7,35.0,0,1,...,0,0,1,0,1,0,0,1,1,0
4,"КГУ ""Общеобразовательная средняя школа имени А...",6,Естествознание и познание мира,Казахский,120506500771,10111000000111010001,9,45.0,1,0,...,0,1,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11337,"КГУ ""Кумжарганская общеобразовательная средняя...",10,Физика,Казахский,081216651095,11010011000001100001,8,40.0,1,1,...,0,0,0,1,1,0,0,0,0,1
11338,КГУ «Общеобразовательная средняя школа имени Ж...,10,Физика,Казахский,081218552048,10011000001100000101,7,35.0,1,0,...,1,1,0,0,0,0,0,1,0,1
11339,КГУ «Жабасакская общеобразовательная средняя ш...,10,Физика,Казахский,081220553394,10000001100000000010,4,20.0,1,0,...,0,0,0,0,0,0,0,0,1,0
11340,"КГУ ""Кызылжарская средняя школа""",10,Физика,Казахский,081222652036,10009001000009000000,2,10.0,1,0,...,0,0,0,0,0,0,0,0,0,0


Assigning regions to each school

In [68]:
regions = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\Все школы.xlsx").rename(columns={
    'Район':'region',
    'Наименование организации':'school'
})

In [69]:
regions = regions.map(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

In [70]:
regions = regions[['region', 'school']]

In [71]:
merged_df = pd.merge(df, regions, on='school', how='left')

In [72]:
schools_region_map = {
    'КГУ "Копинская средняя школа" отдел образования Хромтауского района' : 'Хромтауский район',
    'КГУ "Копинская средняя школа" отдел образования Байганинского района' : 'Байганинский район',
    'КГУ "Общеобразовательная средняя школа имени Абая" отдел образования Темирского района' : 'Темирский район',
    'КГУ «Общеобразовательная средняя школа имени Абая» отдел образования Айтекебийского района' : 'Айтекебийский район'
}

merged_df.loc[(merged_df['school'].isin(schools_region_map.keys())) & (merged_df['region'].isna()), 'region'] = merged_df['school'].map(schools_region_map)

In [73]:
merged_df.isna().values.any()

False

In [74]:
# --------------------------------------- #
school = merged_df['school'].unique().tolist()
grades = merged_df['grade'].unique()
subjects = merged_df['subject'].unique().tolist()
# --------------------------------------- #

In [75]:
grades

array(['6', '8', '10'], dtype=object)

Contingent

In [76]:
contingent_total = merged_df['iin'].nunique()
print(f"Total number of students: {contingent_total}")

Total number of students: 2763


In [77]:
contingent_grade = merged_df.groupby(['grade'])['iin'].nunique().reset_index()
contingent_grade['fraction'] = contingent_grade['iin'].apply(lambda x: x / contingent_total * 100).round(1)
contingent_grade
# print(f"Number of students by grade: {contingent_grade}")

,grade,iin,fraction
0,10,599,21.7
1,6,1126,40.8
2,8,1038,37.6


In [78]:
contingent_grade_lang = merged_df.groupby(['grade', 'lang'])['iin'].nunique().reset_index()
contingent_grade_lang['fraction'] = contingent_grade_lang['iin'].apply(lambda x: x / contingent_total * 100).round(1)
contingent_grade_lang
# print(f"Number of students by grade and language: {contingent_grade_lang}")

,grade,lang,iin,fraction
0,10,Казахский,556,20.1
1,10,Русский,43,1.6
2,6,Казахский,1007,36.4
3,6,Русский,119,4.3
4,8,Казахский,947,34.3
5,8,Русский,91,3.3


In [79]:
contingent_dict = {}
for grade in grades:
    table = merged_df[merged_df['grade'] == grade]
    result_df = table.groupby(['region', 'grade', 'subject', 'lang'])['iin'].nunique().unstack('region')
    contingent_dict[grade] = result_df

# file_path = r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\МКШ\excels\contingent.xlsx"
# with pd.ExcelWriter(file_path) as writer:
#     for grade, dataframe in contingent_dict.items():
#         dataframe.to_excel(writer, sheet_name=grade)

Percent

In [80]:
percent_dict = {}
for grade in grades:
    table = merged_df[merged_df['grade'] == grade]
    result_df = table.groupby('subject')['percent'].mean().round(1)
    percent_dict[grade] = result_df

# file_path = r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\МКШ\excels\percent_subject.xlsx"
# with pd.ExcelWriter(file_path) as writer:
#     for grade, dataframe in percent_dict.items():
#         dataframe.to_excel(writer, sheet_name=grade)

In [81]:
percent_lang_dict = {}
for grade in grades:
    table = merged_df[merged_df['grade'] == grade]
    result_df = table.groupby(['subject', 'lang'])['percent'].mean().round(1)
    percent_lang_dict[grade] = result_df

# file_path = r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\МКШ\excels\percent_subject_lang.xlsx"
# with pd.ExcelWriter(file_path) as writer:
#     for grade, dataframe in percent_lang_dict.items():
#         dataframe.to_excel(writer, sheet_name=grade)

Elements

In [184]:
elem_df = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\elements.xlsx")

elem_subject_map = {
    '8PHYS' : 'Физика', 
    '8BIOL' : 'Биология', 
    '8ENFL' : 'Английский язык', 
    '4SCIE' : 'Естествознание и познание мира', 
    '4MATH' : 'Математика', 
    '8MATH' : 'Математика', 
    '4RUFL' : 'Русский язык (Я1)',
    '8RUFL' : 'Русский язык (Я1)',
    '8RUSL' : 'Русский язык (Я2)', 
    '8CHEM' : 'Химия', 
    '4KZFL' : 'Казахский язык (Я1)', 
    '8KZFL' : 'Казахский язык (Я1)', 
    '8KZSL' : 'Казахский язык (Я2)'
}

elem_df['subject'] = elem_df['subject'].map(elem_subject_map)

In [205]:
elem_melted = merged_df.melt(id_vars=['subject', 'iin'], value_vars=cols)

elem_percent_dict = {}

for grade in grades:
    subject_dict = {}
    for subject in subjects:
        table = merged_df[(merged_df['grade'] == grade) & (merged_df['subject'] == subject)]

        elem_melted = table.melt(id_vars=['subject', 'iin'], value_vars=cols)
        elem_grouped = elem_melted.groupby(['subject', 'variable'])['value'].sum().reset_index().rename(columns={'variable' : 'question'}).assign(percent = lambda x: x['value'] / 20 * 100).round(1)

        if len(elem_grouped) > 0:
            elem_grouped['percent'] = elem_grouped.apply(lambda x: x['value'] / elem_melted[elem_melted['subject'] == subject]['iin'].nunique() * 100, axis=1).round(1)
            elem_percent = elem_grouped.groupby(['subject', 'question'])['percent'].mean().round(1).reset_index().sort_values(by='percent', ascending=False)
            result_df = pd.merge(elem_percent, elem_df, on=['subject', 'question'])
            result_df = result_df[['question', 'element', 'percent']]
            result_df['question'] = range(1, len(result_df) + 1)
            subject_dict[subject] = result_df

    elem_percent_dict[grade] = subject_dict

In [208]:
# file_path = r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\МКШ\excels\elem_percent_tables.xlsx"
# with pd.ExcelWriter(file_path) as writer:
#     for grade, subjects in elem_percent_dict.items():
#         for subject, table in subjects.items():
#             table.to_excel(writer, sheet_name=f"{grade}_{subject}", index=False)

c:\Users\kassenov_r\report-automation\myenv\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
